# Example 1

In [42]:

import os
from openai import AzureOpenAI

# Set environment variables
endpoint = os.getenv("AZURE_OAI_ENDPOINT") 
deployment = "gpt-35-turbo-16k"
api_key = os.getenv("AZURE_OAI_KEY")

# Create AzureOpenAI client
client = AzureOpenAI(azure_endpoint=endpoint, api_key=api_key, api_version="2024-02-01")

# Create chat completion
completion = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "user", "content": "How are you?"}
    ]
)

# Print the response from the AI model
print(completion.choices[0].message.content)


As an AI, I don't have feelings, but I'm functioning properly and ready to assist you with any queries or tasks you have. How can I help you today?


In [46]:
import os
from openai import AzureOpenAI

# Set environment variables
endpoint = os.getenv("AZURE_OAI_ENDPOINT") 
deployment = "gpt-35-turbo-16k"
api_key = os.getenv("AZURE_OAI_KEY")

# Create AzureOpenAI client
client = AzureOpenAI(azure_endpoint=endpoint, api_key=api_key, api_version="2024-02-01")

response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": "Who were the founders of Microsoft?"}
    ]
)
#print(response)
print(response.model_dump_json(indent=2))
print(response.choices[0].message.content)

{
  "id": "chatcmpl-9YZMZo3ApQI8OBcfchv9L7I19UOBG",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Microsoft was founded by Bill Gates and Paul Allen in April 1975. Bill Gates, who is widely known as one of the most successful entrepreneurs in the tech industry, served as the company's chairman and CEO until 2000. Paul Allen, who sadly passed away in 2018, played a significant role in the early development of Microsoft and co-founded the company with Gates.",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
 

## Example 2

In [54]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI


load_dotenv()



# Set environment variables
endpoint = os.getenv("AZURE_OAI_ENDPOINT") 
deployment = "gpt-35-turbo-16k"
api_key = os.getenv("AZURE_OAI_KEY")

# Create AzureOpenAI client
client = AzureOpenAI(azure_endpoint=endpoint, api_key=api_key, api_version="2024-02-01")

completion = client.chat.completions.create(
    model=deployment,
    messages=[
        {
            "role": "user",
            "content": "Who is DRI?",
        },
        {
            "role": "assistant",
            "content": "DRI stands for Directly Responsible Individual of a service. Which service are you asking about?"
        },
        {
            "role": "user",
            "content": "Opinion mining service"
        }
    ],

)


#print(response)
print(completion.model_dump_json(indent=2))
print(completion.choices[0].message.content)


{
  "id": "chatcmpl-9YZe2oiv45D2u4Rqq7OHRZLUf0oPt",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "There doesn't seem to be a commonly known organization or individual referred to as \"DRI\" specifically in relation to an opinion mining service. It's possible that \"DRI\" could be a reference to a company or individual operating in a specific industry or context that provides opinion mining services, but without more specific information, it is difficult to determine the exact entity in question.",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "vio

## Setup Main  Function 


Let us build a simple `main` function that is an entry point for a program that interacts with the Azure OpenAI service. It takes a function as an argument, which will be called with the Azure OpenAI client and model.

H

1. **Initialization**: The function starts by loading environment variables using `load_dotenv()`.
2. **Azure OpenAI Configuration**: The function retrieves three environment variables:
	* `AZURE_OAI_ENDPOINT`: the endpoint URL for the Azure OpenAI service
	* `AZURE_OAI_KEY`: the API key for the Azure OpenAI service
	* `AZURE_OAI_MODEL`: the model identifier for the Azure OpenAI service
3. **Azure OpenAI Client Creation**: The function creates an instance of the `AzureOpenAI` client, passing in the endpoint URL, API key, and API version (`0613`).
4. **Function Call**: The function checks if the passed argument `func` is a callable function using the `callable` function. If it is, the function calls the passed function with the Azure OpenAI client and model as arguments.
5. **Error Handling**: The function catches any exceptions that may occur during execution and prints the error message.

In [55]:

import os
from dotenv import load_dotenv
import utils
# Add OpenAI import. (Added code)
from openai import AzureOpenAI, Model, ChatCompletion
def main(func):
    try:
        load_dotenv()
        utils.initLogFile()
        azure_oai_endpoint = os.getenv("AZURE_OAI_ENDPOINT")
        azure_oai_key = os.getenv("AZURE_OAI_KEY")
        azure_oai_model = os.getenv("AZURE_OAI_MODEL")
        # Define Azure OpenAI client (Add code here)
        client = AzureOpenAI(azure_endpoint=azure_oai_endpoint, api_key=azure_oai_key, api_version="2024-02-01")
        if callable(func):
            func(client, azure_oai_model)
        else:
            print("Invalid input. Please pass a valid function.")

    except Exception as ex:
        print(ex)


### Scenario 1: Implementing a Proof of Concept (PoC) for Azure OpenAI

In this scenario, we're tasked with deploying a GPT-35-turbo-16k model in Azure OpenAI and configuring a sample application to connect to the resources. This serves as our initial PoC to demonstrate the capabilities of Azure OpenAI.

In [56]:
# Task 1: Validate PoC
def function1(aiClient, aiModel):
    inputText = utils.getPromptInput("Task 1: Validate PoC", "sample-text.txt")
    
    # Build messages to send to Azure OpenAI model. (Modified code)
    messages = [
        {"role": "user", "content": inputText}  # Modified to us"user" role and "content" key
    ]
    
    # Define argument list (Modified code)
    apiParams = {
        "model": aiModel,  # Added model parameter
        "messages": messages,
    }
    utils.writeLog("API Parameters:\n", apiParams)
    # Call chat completion connection. (Modified code)

    response = aiClient.chat.completions.create(**apiParams) # Modified to use the aiClient and **apiParams
    utils.writeLog("Response:\n", str(response))
    print("Response: " + response.choices[0].message.content + "\n")
    return response

In [58]:
# Call the main function with function1 as an argument
main(function1)

Would you like to type a prompt or text in file sample-text.txt? (type/file)
...Sending the following request to Azure OpenAI...
Request: hello

Response: Hello! How can I assist you today?



### Scenario 2: Developing the PoC App for Company Chatbot

Here, we'll further develop the PoC app to explore the potential of Azure OpenAI for company chatbot functionality. The goal is to develop an app that provides responses in a casual tone, limited to 1,000 tokens, and with a temperature of 0.5.

In [59]:
# Task 2: Company chatbot
def function2(aiClient, aiModel):
    inputText = utils.getPromptInput("Task 2: Company chatbot", "sample-text.txt")
    
    # Build messages to send to Azure OpenAI model. (Added code)
    messages = [
        {"role": "user", "content": inputText},
        {"role": "assistant", "content": "You can find it on the footer of every page on our website. Hope that helps! Thanks for using Contoso, Ltd."}
    ]
    
    # Define argument list (Modified code)
    apiParams = {
        "model": aiModel,
        "messages": messages,
        "temperature": 0.5,
        "max_tokens": 1000
    }
    
    utils.writeLog("API Parameters:\n", apiParams)

    # Call chat completion connection. (Modified code)
    response = aiClient.chat.completions.create(**apiParams) # Modified to use the aiClient and **apiParams
    utils.writeLog("Response:\n", str(response))
    print("Response"+ response.choices[0].message.content + "\n")
    return response

In [60]:
# Call the main function with function1 as an argument
main(function2)

Would you like to type a prompt or text in file sample-text.txt? (type/file)
...Reading text from sample-text.txt...


...Sending the following request to Azure OpenAI...
Request: What can a generative AI model do? Give me a short answer.

ResponseA generative AI model can create new content, such as text, images, or music, based on patterns and examples it has been trained on.



### Scenario 3: Using the PoC App for Developer Tasks

In this scenario, we'll use the PoC app to assist with developer tasks such as code refactoring and unit testing. This demonstrates how Azure OpenAI can enhance productivity and streamline development workflows.

In [61]:
# Task 3: Developer tasks
def function3(aiClient, aiModel):
    inputText = utils.getPromptInput("Task 3: Developer tasks", "sample-text.txt")
    
    # Build messages to send to Azure OpenAI model. (Added code)
    messages = [
        {"role": "user", "content": inputText},
        {"role": "assistant", "content": ""}  # Initialize response content
    ]
    
    # Define argument list (Modified code)
    apiParams = {
        "model": aiModel,
        "messages": messages,
        "temperature": 0.5,  # Set temperature to 0.5
        "max_tokens": 1000  # Set max tokens to 1000
    }
    
    utils.writeLog("API Parameters:\n", apiParams)

    # Call chat completion connection. (Modified code)
    response = aiClient.chat.completions.create(**apiParams) # Modified to use the aiClient and **apiParams
    
    utils.writeLog("Response:\n", str(response))
    print("Response: " + response.choices[0].message.content + "\n")
    return response

In [62]:
# Call the main function with function1 as an argument
main(function3)

Would you like to type a prompt or text in file sample-text.txt? (type/file)
...Reading text from sample-text.txt...


...Sending the following request to Azure OpenAI...
Request: What can a generative AI model do? Give me a short answer.

Response: A generative AI model can create new and original content, such as text, images, music, and even videos, based on patterns and examples it has learned from.



### Scenario 4: Using Company Data for Travel Recommendations

Finally, we'll extend the PoC app to utilize our company's data to better answer customer questions related to travel. The goal is to connect the PoC app to an Azure AI Search resource that contains sample travel data, providing more accurate and relevant responses.

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
import os
from dotenv import load_dotenv
from openai import AzureOpenAI


load_dotenv()
search_api_key = os.getenv("SEARCH_API_KEY") #"<your_search_api_key>"
blob_storage_connection_string =os.getenv("CONNECTION_STRING") #"<your_blob_storage_connection_string>"
def function4(aiClient, aiModel):
    # Assuming you have the necessary Azure SDK installed and configured

    # Load configuration file or define connection strings
    search_service_name = "search41431948"
    index_name = "pocindex"
    search_api_key =search_api_key #"<your_search_api_key>"
    blob_container_name = "sa41431948"
    blob_storage_connection_string =blob_storage_connection_string #"<your_blob_storage_connection_string>"

    # Initialize Azure Search client
    search_client = SearchClient(account_url=f"https://{search_service_name}.search.windows.net/",
                                 index_name=index_name,
                                 credential=AzureKeyCredential(search_api_key))

    inputText = utils.getPromptInput("Task 4: Use company data", "sample-text.txt")

    # Build messages to send to Azure OpenAI model
    messages = [
        {"role": "user", "content": inputText}
    ]

    # Define argument list
    apiParams = {
        "model": aiModel,
        "messages": messages,
        "temperature": 0.5,  # Set temperature to 0.5
        "max_tokens": 1000  # Set max tokens to 1000
    }

    utils.writeLog("API Parameters:\n", apiParams)

    # Call chat completion connection.
    response = aiClient.chat.completions.create(**apiParams)
    utils.writeLog("Response:\n", str(response))

    # Extract the user's query
    user_query = response.choices[0].message.content

    # Perform search on Azure AI Search
    search_result = search_client.search(search_text=user_query, top=1)

    if search_result:
        # Extract the first result
        first_result = search_result[0]

        # Extract the system message
        system_message = "You are a helpful travel agent."

        # Build the response message
        response_message = {
            "role": "system",
            "content": f"{system_message} {first_result['your_desired_field']}"  # Update 'your_desired_field' to your desired field from the search result
        }

        # Append the response message to the messages list
        messages.append(response_message)

        # Update apiParams with new messages
        apiParams['messages'] = messages

        # Call chat completion connection again with updated messages
        response = aiClient.chat.completions.create(**apiParams)
        utils.writeLog("Response:\n", str(response))
        print("Response: " + response.choices[0].message.content + "\n")
    else:
        print("No search result found.")

    return response


In [64]:
# Call the main function with function1 as an argument
main(function4)

Would you like to type a prompt or text in file sample-text.txt? (type/file)
...Reading text from sample-text.txt...


...Sending the following request to Azure OpenAI...
Request: What can a generative AI model do? Give me a short answer.

No module named 'azure'


Call Azure OpenAI resource
Once you've configured your connection to Azure OpenAI, send your prompt to the model.

In [6]:
from openai import AzureOpenAI
import os
from dotenv import load_dotenv
load_dotenv()
azure_oai_endpoint = os.getenv("AZURE_OAI_ENDPOINT")
azure_oai_key = os.getenv("AZURE_OAI_KEY")
azure_oai_model = os.getenv("AZURE_OAI_MODEL")
from openai import AzureOpenAI
deployment_name = 'tz-edu-gpt-35-turbo' 
# Initialize the Azure OpenAI client
client = AzureOpenAI(
        azure_endpoint = azure_oai_endpoint, 
        api_key=azure_oai_key,  
        api_version="2024-02-01" #  Target version of the API, such as 2024-02-15-preview
        )
response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is Azure OpenAI?"}
    ]
)
generated_text = response.choices[0].message.content
# Print the response
print("Response: " + generated_text + "\n")

In [ ]:
# Task 4: Use company data
def function4(aiClient, aiModel):
    inputText = utils.getPromptInput("Task 4: Use company data", "sample-text.txt")
    
    # Build messages to send to Azure OpenAI model. (Added code)
    messages = [
        {"role": "system", "content": "You are a helpful travel agent."},
        {"role": "user", "content": inputText}
    ]
    
    # Define connection and argument list (Added code)
    from azure.ai.search import SearchClient
    from azure.ai.search.models import QueryType
    search_client = SearchClient("https://search41431948.search.windows.net/", "pocindex")
    apiParams = {
        "search_client": search_client,
        "query": inputText,
        "query_type": QueryType.KEYWORD
    }
    
    utils.writeLog("API Parameters:\n", apiParams)
    
    # Call search connection. (Added code)
    response = search_client.search(**apiParams)
    
    utils.writeLog("Response:\n", str(response))
    print("Response: " + response.results[0].text + "\n")
    return response